In [ ]:
!pip install -U transformers datasets scipy scikit-learn ipywidgets

In [36]:
import random
from IPython.display import display, HTML

import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator

In [4]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
task = "mrc"
model_checkpoint = "klue/roberta-base"
batch_size = 16

In [6]:
datasets = load_dataset("klue", task)

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/mrc/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90. Subsequent calls will reuse this data.


In [7]:
datasets

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'guid', 'is_impossible', 'news_category', 'question', 'question_type', 'source', 'title'],
        num_rows: 17554
    })
    validation: Dataset({
        features: ['answers', 'context', 'guid', 'is_impossible', 'news_category', 'question', 'question_type', 'source', 'title'],
        num_rows: 5841
    })
})

In [8]:
datasets["train"][0]

{'answers': {'answer_start': [478, 478], 'text': ['한 달가량', '한 달']},
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
 'guid': 'klue-mrc-v1_train_12759',
 'is_impossible': False,
 'news_category': '종합',
 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
 'question_type': 1,
 'source': 'hankyung',
 'title

In [12]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [13]:
show_random_elements(datasets["train"])

,answers,context,guid,is_impossible,news_category,question,question_type,source,title
0,"{'answer_start': [122], 'text': ['애플']}","중국 스타벅스에서 카페라테 한 잔을 마시려면 미국에서보다 약 1달러 더 비싼 4.80달러(약 5237원)를 내야 한다. 미국에서 7만3000달러인 캐딜락 에스컬레이드는 중국에서 22만9000달러(약 2억5000만원), 애플 아이패드2는 중국에서 미국보다 100달러 더 비싼 488달러(약 53만원)다. 왜 이런 현상이 벌어질까.월스트리트저널(WSJ)은 중국 ‘살인 물가’의 원인이 높은 세금과 중국 중산층의 과시욕 때문이라고 4일(현지시간) 보도했다. 중국 중산층이 급격히 늘면서 수입품을 선호하는 소비자가 늘었고, 가격이 비쌀수록 더 잘 팔리는 중국 시장을 겨냥해 외국 브랜드들이 가격을 더 높게 책정해왔다는 것이다.스타벅스가 대표적이다. 미국에서는 대중적인 커피 전문점이지만 중국에선 고급 이미지로 자리 잡았다. 중간 크기의 아메리카노 한 잔 가격은 미국보다 75% 비싸게 팔린다. 유벌 애츠먼 맥킨지앤드컴퍼니 사장은 “중국 소비자는 가격이 비쌀수록 품질이 뛰어나고, 명품을 소비하면 자신도 명품이 된다고 생각하는 경향이 있다”며 “많은 외국계 기업이 이런 소비자의 성향을 겨냥해 많은 수익을 내왔다”고 말했다. 중국 정부의 복잡한 행정 절차가 소비자 물가를 올리는 원인이라는 지적도 있다. 법률회사 캐드월더러 워커셤앤드태프트의 중국 담당자인 로키 리는 “중국에서 매장 하나를 열기 위해선 여러 단계의 복잡한 행정 절차를 걸쳐 정부의 허가를 받아야 하기 때문에 적게는 몇 달에서 길게는 몇 년이 걸린다”며 “이런 비용이 모두 소비자 부담으로 이어지고 있는 것”이라고 지적했다.WSJ는 중국에서만 비싸게 받던 외국 기업들의 관행은 지속되기 힘들 것이라고 전망했다. 인터넷이 널리 보급되고 중국인의 해외 여행이 급증하면서 소비자가 쉽게 가격 비교를 하고 있기 때문이다.",klue-mrc-v1_train_13642,True,국제,중국에서 가격 할인을 하는 외국 회사는?,3,hankyung,中 스타벅스 커피값이 美보다 1달러 비싼 까닭 …
1,"{'answer_start': [139], 'text': ['수양대군']}","조선 세종의 셋째 아들인 안평대군(1418∼1453)의 글씨로, 말아서 둘 수 있게 만든 형태이다.\n\n안평대군은 어려서부터 학문을 좋아하고 시, 글씨, 그림에 모두 능하였으나 특히 글씨에 능하였다. 당시 복잡한 정권 다툼에 말려들어 젊은 나이에 수양대군에게 죽임을 당하였고, 죽은 뒤에는 그의 글씨가 불태워져 현재 확인할 수 있는 것은 비문이나 글씨교본 그리고 일본에 있는 〈몽유도원도〉의 발문과 〈소원화개첩〉에서 뿐이다.\n\n〈소원화개첩〉은 가로 16.5 cm, 세로 26.5cm로 비단 위에 행서체로 쓰여진 것으로 모두 56자가 들어 있다. 글 끝에 낙관과 도장이 구비되어 있다. 원래의 시는 당나라 시인인 이상은이 쓴 것이나 원문과 다른 곳이 몇 군데 있다. 비록 소품이나 행서체를 대표하는 작품으로 조맹부체의 영향을 받았으나 누구도 따를 수 없는 웅장하고 활달한 그의 기품이 잘 나타나 있다. 전형적인 그의 서풍이 잘 나타나 있는 작품으로 낙관과 도장이 찍혀있어 안평대군의 진본임이 확실한, 국내에서 지금까지 발견된 그의 유일한 작품으로 귀중한 가치가 있다.",klue-mrc-v1_train_01177,True,None,안평대군이 정권 싸움에 휘말리면서 젊은 나이에 죽였던 인물은 누구인가?,3,wikipedia,소원화개첩
2,"{'answer_start': [111], 'text': ['150석']}","1996년 국회의원 총선거 결과 신한국당이 139석, 새정치국민회의가 79석, 자유민주연합이 50석, 통합민주당이 15석, 무소속은 16석을 차지하였다. 이로써 집권 여당인 신한국당은 또다시 과반인 150석 달성에 실패한 것으로 나타났다.\n\n이후 신한국당은 통합민주당에서 당명을 바꾼 민주당 측 당선자 15명과 무소속 당선자 16명 등을 상대로 포섭에 들어갔고, 이같은 방식으로 신한국당이 개원을 앞두고 과반을 살짝 넘는 151석 확보에 성공하자 야당은 인위적인 국회 장악이며 민심 왜곡이라고 비판하였다. 신한국당은 자진해서 입당하는 사람들을 막지 않은 것 뿐이라는 입장이었으나, 야당은 신한국당이 대대적인 회유와 협박, 공권력을 동원한 외압을 통해 이들을 입당시켰으므로 소신에 따른 자발적 입당이라고 할 수 없다고 비판하였다. \n\n이런 상황에서 국회법에 규정된 개원 시한인 6월 5일까지도 여야가 원 구성 등에 대한 합의를 마치지 못하자, 신한국당은 준법 개원을 주장하며 6월 5일 단독으로라도 본회의를 개최하고 국회의장단 선거를 실시하겠다고 선언하였다. 국민회의, 자민련 등 야당은 국회법의 개원 시한 규정은 강제 규정이 아닌 훈시 규정이라며 원 구성 협상이 타결된 후에 개원을 해야 한다고 주장하였다. 결국 신한국당은 6월 5일 단독으로 본회의를 열었으나, 원내 최연장자로써 정식 의장 선출 전까지 임시의장을 맡은 김허남 자민련 의원이 기습적으로 본회의를 산회시켰고, 국회의장단 선거는 실시되지 못하였다. \n\n6월 12일 다시 열린 본회의에서 신한국당은 다시 의장단 선거를 상정할 것을 주장하였으며, 야당 의원들은 이에 반대해 긴 시간 토론이 진행되었다. 본회의가 4시간 가량 이어지자 김허남 임시의장은 피로를 호소하며 정회를 선포하였으며, 차연장자인 김명윤 신한국당에게 임시의장직을 양보하고 귀가하였다. 이후 김명윤 임시의장은 본회의를 재개하고 의장단 선거를 실시하고자 하였으나, 야당 의원들의 저지로 의장석에 오르지도 못하게 되자 정회를 선포하였다. 결국 이 날 본회의는 의장단을 선출하지 못하고 산회되었다.\n\n국회는 두 차례 휴회 끝에 6월 24일 김명윤 임시의장의 사회로 다시 본회의를 열었는데, 신한국당은 본회의가 정회된 후 야당 의원들이 자리를 비우자 기습적으로 본회의를 재개, 의장단 선출을 시도하였다. 그러자 본회의장에 돌아온 야당 의원들은 기표소와 투표함, 명패함을 점거하고 투표지를 탈취하며 선거를 방해하였고, 결국 의장단 선출은 또다시 무산되었다. \n\n이후 원내 교섭단체들 간 합의가 타결됨에 따라 국회의장단 선거는 임시 국회 회기 마지막 날인 7월 4일 바야흐로 치러질 수 있었다. 그러나 원 구성 협상 과정에서 배제된 것에 반발한 민주당이 의장석을 점거하고 농성 김허남 임시의장은 발언대에서 의장 선거를 진행해야 했다. 의장 선거 직후 민주당 의원들은 의사진행발언을 하게 해주는 조건으로 농성을 해제하였으며, 따라서 신임 김수한 의장은 의장석에서 본회의를 진행할 수 있었다.",klue-mrc-v1_train_00210,False,None,신한국당이 의석 과반수를 차지하기 위해 필요했던 의석 수는?,1,wikipedia,대한민국 제15대 국회 전반기 의장단 선거
3,"{'answer_start': [836], 'text': ['13.8㎞/L']}","지난 2일 출시된 현대자동차 쏘나타 1.7 디젤의 연비가 상반기 수입차 베스트셀러 3위에 오른 폭스바겐(독일) 골프 2.0 TDI(디젤)보다 높은 것으로 나타났다. 골프 2.0 TDI가 오는 9월부터 적용되는 디젤차 배기가스 기준인 ‘유로6’를 맞추기 위해 신모델 연비를 L당 15.5㎞로 수정하면서 쏘나타 디젤(최고 16.8㎞/L)에 역전당했다. 이런 현상은 소형 해치백, 중형 세단, 중소형 스포츠유틸리티차량(SUV) 등 주요 차급에서도 나타나고 있다. 수입 디젤차들은 잇따라 연비를 하향 조정하는 반면 국산 디젤차들은 기술 향상에 힘입어 연비를 높이고 있기 때문이다.○유로6 연비 수입차↓ 국산차 ↑에너지관리공단에 따르면 상반기 수입차 베스트셀러 3위(3395대)에 오른 폭스바겐(독일) 골프 2.0 TDI는 이달 1일자로 연비를 15.5㎞/L로 신고했다. 기존 16.7㎞/L보다 7.1% 내렸다. 폭스바겐은 골프 1.6 TDI의 연비도 18.9㎞/L에서 16.1㎞/L로 낮춰 등록했다.4월 나온 폭스바겐 폴로는 배기량을 1.6L에서 1.4L로 낮췄는데도 연비

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [15]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [2, 28868, 11376, 24852, 7746, 35, 3, 21070, 7746, 11376, 55, 26342, 2045, 10134, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [18]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > max_length:
        break
example = datasets["train"][i]

In [19]:
len(tokenizer(example["question"], example["context"])["input_ids"])

391

In [20]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

In [21]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [22]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 156]

In [23]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP] 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다. 17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2 ~ 3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다. 장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20 ~ 21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24 ~ 25일이었으며 장마기간은 32일, 강수일수는 17. 2일이었다. 기상청은 올해 장마기간의 평균 강수량이 350 ~ 400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 [SEP]
[CLS] 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은? [SEP] 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24 ~ 25일이었으며 장마기간은 32일, 강수일수는 17. 2일이었다. 기상청은 올해 장마기간의 평균 강수량이 350 ~ 400㎜로 평년과 비슷하거나 적을 것으로

In [24]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 1), (1, 4), (5, 7), (7, 8), (9, 11), (11, 12), (12, 13), (13, 14), (15, 17), (17, 18), (19, 21), (22, 24), (24, 25), (26, 29), (29, 30), (31, 33), (33, 34), (34, 35), (0, 0), (0, 1), (1, 3), (4, 6), (6, 7), (8, 10), (10, 11), (12, 15), (15, 17), (18, 20), (20, 21), (21, 22), (22, 23), (24, 26), (27, 28), (29, 31), (31, 33), (33, 34), (35, 37), (37, 38), (38, 39), (40, 42), (42, 43), (44, 46), (47, 48), (48, 49), (50, 52), (53, 54), (54, 55), (56, 58), (58, 59), (60, 62), (62, 63), (64, 66), (66, 68), (68, 69), (69, 71), (71, 72), (73, 76), (76, 77), (78, 80), (80, 81), (82, 85), (86, 88), (89, 90), (90, 92), (92, 93), (94, 95), (95, 96), (97, 99), (99, 100), (100, 101), (101, 102), (103, 105), (105, 107), (108, 110), (111, 114), (115, 117), (118, 119), (120, 122), (122, 124), (124, 125), (126, 128), (128, 130), (130, 131), (131, 132), (133, 136), (136, 138), (139, 141), (141, 142), (143, 146), (146, 147), (147, 148), (149, 151), (151, 152), (152, 153), (154, 155), (155, 15

In [25]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

북 북


In [26]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [27]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

260 263


In [28]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

한 달가량
한 달가량


In [29]:
pad_on_right = tokenizer.padding_side == "right"

In [30]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [31]:
features = prepare_train_features(datasets['train'][:5])

In [32]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

In [34]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a d

In [35]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [37]:
data_collator = default_data_collator

In [38]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [39]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [40]:
trainer.save_model("test-squad-trained")

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


RuntimeError: CUDA error: device-side assert triggered

In [41]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

RuntimeError: CUDA error: device-side assert triggered

In [42]:
output.start_logits.shape, output.end_logits.shape

NameError: name 'output' is not defined

In [43]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

NameError: name 'output' is not defined

In [44]:
n_best_size = 20

In [45]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

NameError: name 'output' is not defined

In [46]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [47]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

KeyError: 'id'

In [48]:
raw_predictions = trainer.predict(validation_features)

NameError: name 'validation_features' is not defined

In [49]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

NameError: name 'validation_features' is not defined

In [50]:
max_answer_length = 30

In [51]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

NameError: name 'output' is not defined

In [52]:
datasets["validation"][0]["answers"]

{'answer_start': [666, 666], 'text': ['뉴 740Li 25주년 에디션', '뉴 740Li 25주년']}

In [53]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

NameError: name 'validation_features' is not defined

In [54]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [55]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

NameError: name 'validation_features' is not defined

In [56]:
metric = load_metric("squad")

In [57]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

NameError: name 'final_predictions' is not defined